In [5]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotnine as gg
from cytominer_eval import evaluate
from pycytominer.cyto_utils import infer_cp_features

In [6]:
data_dir = '2016_04_01_a549_48hr_batch1_consensus_modz_feature_select_dmso.csv.gz'

In [7]:
df = pd.read_csv(data_dir)

In [12]:
features = infer_cp_features(df)
meta_features = infer_cp_features(df, metadata=True)
len(df.Metadata_broad_sample.unique())

1571

In [13]:
ls = df.Metadata_moa.unique()[:2].tolist()
print(ls)
selected_moa= df.query("Metadata_moa==@ls")

['glucocorticoid receptor antagonist', 'calmodulin antagonist']


In [14]:
precision_recall_at_k = []

for k in [2, 10]:
    precision_recall_results = evaluate(
        profiles=selected_moa,
        features=features,
        meta_features=meta_features,
        replicate_groups=["Metadata_moa"],
        operation="precision_recall",
        similarity_metric="pearson",
        precision_recall_k=k
    )
    precision_recall_at_k.append(precision_recall_results)

In [15]:
df_res_1 = pd.concat(precision_recall_at_k).reset_index(drop=True)
print(df_res_1.shape)
df_res_1.head()

(4, 4)


,Metadata_moa,k,precision,recall
0,calmodulin antagonist,2.0,0.0,0.000000
1,glucocorticoid receptor antagonist,2.0,0.0,0.000000
2,calmodulin antagonist,10.0,0.4,0.030303
3,glucocorticoid receptor antagonist,10.0,0.2,0.066667


In [16]:
pr_av = []
for k in df_res_1.k.unique():
    pr_av.append(df_res_1.query("k==@k").mean().tolist())
pr_av

[[2.0, 0.0, 0.0], [10.0, 0.30000000000000004, 0.048484848484848485]]